# AI - Project 01- Mohsen Fayyaz - 810196650

# BFS)
<b>Initial State:</b> map read from the file
<br>

<b>Goal State:</b> a map which doesn't contain any numbers in it i.e. 1,2 or 3.
<br>

<b>actions:</b> There are 8 different possible actions in any state which are moving P or Q one block to one of the four cardinal directions. (The move might not be possible due to colliding with walls or etc.)


For any new state we make, we add another node to frontier and also the node's hash into explored set. That's because in BFS we can be sure that if we see a new state which was added into the frontier or explored set before, it doesn't have to be added to the frontier again.(This assumption is not true for A* algorithm)


In [1]:
from State import State
from queue import Queue
from time import sleep
import copy
from os import system, name
import sys
from IPython.display import clear_output

P_CHAR = "P"
Q_CHAR = "Q"
WALL_CHAR = "%"
EMPTY_CHAR = " "
P_FOOD = "1"
Q_FOOD = "2"
BOTH_FOOD = "3"


class Pac_map_handler:
    @staticmethod
    def find_in_map(the_map, char):
        for line in the_map:
            if char in line:
                return the_map.index(line), line.index(char)

    @staticmethod
    def is_in_map(pac_map: list, row, col):
        return 0 <= row < len(pac_map) and 0 <= col < len(pac_map[0])

    @staticmethod
    def can_p_goto(pac_map: list, row, col):
        return Pac_map_handler.is_in_map(pac_map, row, col) and pac_map[row][col] in {P_FOOD, EMPTY_CHAR, BOTH_FOOD}

    @staticmethod
    def can_q_goto(pac_map: list, row, col):
        return Pac_map_handler.is_in_map(pac_map, row, col) and pac_map[row][col] in {Q_FOOD, EMPTY_CHAR, BOTH_FOOD}

    # P MOVEMENT
    @staticmethod
    def move_p_right(pac_map, row, col):
        if Pac_map_handler.can_p_goto(pac_map, row, col + 1):
            pac_map[row][col] = EMPTY_CHAR
            pac_map[row][col + 1] = P_CHAR
        return pac_map

    @staticmethod
    def move_p_left(pac_map, row, col):
        if Pac_map_handler.can_p_goto(pac_map, row, col - 1):
            pac_map[row][col] = EMPTY_CHAR
            pac_map[row][col - 1] = P_CHAR
        return pac_map

    @staticmethod
    def move_p_up(pac_map, row, col):
        if Pac_map_handler.can_p_goto(pac_map, row - 1, col):
            pac_map[row][col] = EMPTY_CHAR
            pac_map[row - 1][col] = P_CHAR
        return pac_map

    @staticmethod
    def move_p_down(pac_map, row, col):
        if Pac_map_handler.can_p_goto(pac_map, row + 1, col):
            pac_map[row][col] = EMPTY_CHAR
            pac_map[row + 1][col] = P_CHAR
        return pac_map

    # Q MOVEMENT
    @staticmethod
    def move_q_right(pac_map, row, col):
        if Pac_map_handler.can_q_goto(pac_map, row, col + 1):
            pac_map[row][col] = EMPTY_CHAR
            pac_map[row][col + 1] = Q_CHAR
        return pac_map

    @staticmethod
    def move_q_left(pac_map, row, col):
        if Pac_map_handler.can_q_goto(pac_map, row, col - 1):
            pac_map[row][col] = EMPTY_CHAR
            pac_map[row][col - 1] = Q_CHAR
        return pac_map

    @staticmethod
    def move_q_up(pac_map, row, col):
        if Pac_map_handler.can_q_goto(pac_map, row - 1, col):
            pac_map[row][col] = EMPTY_CHAR
            pac_map[row - 1][col] = Q_CHAR
        return pac_map

    @staticmethod
    def move_q_down(pac_map, row, col):
        if Pac_map_handler.can_q_goto(pac_map, row + 1, col):
            pac_map[row][col] = EMPTY_CHAR
            pac_map[row + 1][col] = Q_CHAR
        return pac_map

    @staticmethod
    def print_map(pac_map):
        print('\n'.join(''.join(item) for item in pac_map))
    


In [2]:
class State:
    def __init__(self, map_data, parent=None):
        self.map_data = map_data
        self.parent = parent
        self.hash = ''.join(''.join(item) for item in map_data).replace("%", "")

    def get_hash(self):
        return self.hash

    def get_map(self):
        return self.map_data

    def get_parent(self):
        return self.parent


## BFS implementation:

In [13]:
class BFS:
    def __init__(self, pac_map):
        self.start_state = State(pac_map, None)
        self.explored_states_hash = set()
        self.unique_states_hash = set()
        self.num_of_explored_states = 0
        self.num_of_unique_explored_states = 0
        self.frontier_states = Queue()
        self.goal_state = None
        self.goal_state_is_found = False

    def start(self):
        self.goal_state_is_found = False
        self.frontier_states = Queue()
        self.frontier_states.put(self.start_state)
        self.explored_states_hash = set()
        self.explored_states_hash.add(self.start_state.get_hash())
        self.unique_states_hash = set()
        self.num_of_explored_states = 0
        self.num_of_unique_explored_states = 0
        while not self.frontier_states.empty():
            current_state = self.frontier_states.get()
            self.num_of_explored_states += 1
            if current_state.get_hash() not in self.unique_states_hash:
                self.unique_states_hash.add(current_state.get_hash())
                self.num_of_unique_explored_states += 1

            # self.print_map(current_state.get_map())
            if self.are_constraints_satisfied(current_state):
                self.goal_state = current_state
                self.goal_state_is_found = True
                return True

            self.do_actions(current_state)
            if self.goal_state_is_found:
                return True

        return False

    def print_solution(self):
        if self.goal_state is None:
            print("No Solution!")
        else:
            current_state = self.goal_state
            goal_depth = 0
            states_list = list()
            while current_state is not None:
                goal_depth += 1
                states_list.append(current_state)
                current_state = current_state.get_parent()

            for state in reversed(states_list):
                cls()
                Pac_map_handler.print_map(state.get_map())
                sleep(0.2)

            print("Explored States: " + str(self.num_of_explored_states))
            print("Explored Unique States: " + str(self.num_of_unique_explored_states))
            print("Goal Depth: " + str(goal_depth))

    def are_constraints_satisfied(self, state: State):
        for line in state.get_map():
            if P_FOOD in line or Q_FOOD in line or BOTH_FOOD in line:
                return False
        return True

    def do_actions(self, current_state):
        pac_map = current_state.get_map()
        p_row, p_col = Pac_map_handler.find_in_map(pac_map, P_CHAR)
        q_row, q_col = Pac_map_handler.find_in_map(pac_map, Q_CHAR)
        # P MOVEMENT
        self.move_p(current_state, pac_map, p_row, p_col)
        # Q MOVEMENT
        self.move_q(current_state, pac_map, q_row, q_col)

    def move_p(self, current_state, pac_map, row, col):
        new_map = Pac_map_handler.move_p_up(copy.deepcopy(pac_map), row, col)
        self.add_to_frontier(new_map, current_state)

        new_map = Pac_map_handler.move_p_left(copy.deepcopy(pac_map), row, col)
        self.add_to_frontier(new_map, current_state)

        new_map = Pac_map_handler.move_p_down(copy.deepcopy(pac_map), row, col)
        self.add_to_frontier(new_map, current_state)

        new_map = Pac_map_handler.move_p_right(copy.deepcopy(pac_map), row, col)
        self.add_to_frontier(new_map, current_state)

    def move_q(self, current_state, pac_map, row, col):
        new_map = Pac_map_handler.move_q_up(copy.deepcopy(pac_map), row, col)
        self.add_to_frontier(new_map, current_state)

        new_map = Pac_map_handler.move_q_left(copy.deepcopy(pac_map), row, col)
        self.add_to_frontier(new_map, current_state)

        new_map = Pac_map_handler.move_q_down(copy.deepcopy(pac_map), row, col)
        self.add_to_frontier(new_map, current_state)

        new_map = Pac_map_handler.move_q_right(copy.deepcopy(pac_map), row, col)
        self.add_to_frontier(new_map, current_state)

    def add_to_frontier(self, new_map, current_state):
        new_state = State(new_map, current_state)
        if not new_state.get_hash() in self.explored_states_hash:
            self.frontier_states.put(new_state)
            self.explored_states_hash.add(new_state.get_hash())
            if self.are_constraints_satisfied(new_state):
                self.goal_state = new_state
                self.goal_state_is_found = True


def cls():
    # for windows
    if name == 'nt':
        _ = system('cls')

        # for mac and linux(here, os.name is 'posix')
    else:
        _ = system('clear')

    clear_output(wait=True)


In [14]:
def main():
    with open('test3', 'r') as file:
        pac_map = file.read().splitlines()

    characterized_map = list()
    for line in pac_map:
        characterized_map.append(list(line))

    print("Loading...")
    start = time()

    my_bfs = BFS(characterized_map)
    my_bfs.start()
    my_bfs.print_solution()

    end = time()
    print("Time: " + str(end - start) + "s")


main()

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%            Q   %      %%%% %   %%  %%% %      %
% %%%%% %% %%%     %  %     P          %  %%%%%%%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Explored States: 2372
Explored Unique States: 2372
Goal Depth: 21
Time: 7.835578203201294s


<h2>Zebra stripes, footer</h2>
<table style="width:100%" >
    <tr>
        <td>
            <table style="width:100%">
                <thead>
                <tr>
                    <th>#</th>        
                    <th>IMDB Top 10 Movies</th>
                    <th>Year</th>
                </tr>
                </thead>
                <tfoot>
                <tr>
                    <td>&nbsp;</td>        
                    <td></td>
                    <td></td>
                </tr>
                </tfoot>    
                <tr>
                    <td>1</td>        
                    <td>The Shawshank Redemption</td>
                    <td>1994</td>
                </tr>        
                <tr>
                    <td>2</td>         
                    <td>The Godfather</td>
                    <td>1972</td>
                </tr>
                <tr>
                    <td>3</td>         
                    <td>The Godfather: Part II</td>
                    <td>1974</td>
                </tr>    
                <tr>
                    <td>4</td> 
                    <td>The Good, the Bad and the Ugly</td>
                    <td>1966</td>
                </tr>
            </table>
        </td>
        <td>
            <table style="width:100%">
                <thead>
                <tr>
                    <th>#</th>        
                    <th>IMDB Top 10 Movies</th>
                    <th>Year</th>
                </tr>
                </thead>
                <tfoot>
                <tr>
                    <td>&nbsp;</td>        
                    <td></td>
                    <td></td>
                </tr>
                </tfoot>    
                <tr>
                    <td>1</td>        
                    <td>The Shawshank Redemption</td>
                    <td>1994</td>
                </tr>        
                <tr>
                    <td>2</td>         
                    <td>The Godfather</td>
                    <td>1972</td>
                </tr>
                <tr>
                    <td>3</td>         
                    <td>The Godfather: Part II</td>
                    <td>1974</td>
                </tr>    
                <tr>
                    <td>4</td> 
                    <td>The Good, the Bad and the Ugly</td>
                    <td>1966</td>
                </tr>
            </table>
        </td>
    </tr>
</table>

## b)

As shown above <b>OverallQual</b> and <b>SalePrice</b> has a linear relationship
So the linear estimation of the data distribution is calculated below.
Then RMSE of the data is being calculated as written in the project.

<table style="width: 100%">
    <tr>
        <td>
            <img src="http://mohsenfayyaz.ir/extra/files/line.png" alt="line.png" style="width: 100%;"/> -->
        </td>
        <td>
            $$ \huge \hat{y} = wx + b $$ <br>
            $$ w = \frac{\Delta{y}}{\Delta{x}} = \frac{200-(-50)}{6.5-0} = 38.4615 $$ <br>
            $$ b = -50 $$ <br>
            <b> $$ \huge \hat{y} = 38.4615 \times x -50 $$ </b>
        </td>
    </tr>
</table>